# Chat Completions Part 5

Make sure you have an environment variable called OPENAI_API_KEY set with your API key.

## tools (array)
A list of tools the model may call. Currently, only functions are supported as a tool. Use this to provide a list of functions the model may generate JSON inputs for. A max of 128 functions are supported.



In [12]:
import openai
import os
import json

# Define a function to get the current weather for a specified location and unit.
def get_current_weather(location, unit="fahrenheit"):
    """Fetch the current weather for a given location in a specified unit."""
    print(f"Getting weather for {location} in {unit}...")
    return f"The weather in {location} is sunny and 72 degrees."

# Retrieve the OpenAI API key from an environment variable.
api_key = os.environ.get("OPENAI_API_KEY")

# Check if the API key is set in the environment, and raise an error if not.
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

# Create an OpenAI client with the retrieved API key.
client = openai.OpenAI(api_key=api_key)

# Define a list of tools to be used with the API, in this case, a weather function.
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location"],
            },
        }
    }
]

# Set up a user query regarding the weather in a specific location.
messages = [{"role": "user", "content": "What is the weather like in Houston today?"}]

# Get a completion from the model using the defined tools.
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=messages,
    tools=tools,
)

# Check if any function was called during the completion process.
if completion.choices[0].message.tool_calls:
    print("A function call was made.")
    # Extract the function call details.
    tool_call = completion.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    arguments_json = tool_call.function.arguments

    # Convert JSON string of arguments into a dictionary.
    arguments = json.loads(arguments_json)

    # Execute the function using the arguments and print the result.
    if function_name == "get_current_weather":
        weather_result = get_current_weather(**arguments)
        print(weather_result)
    else:
        print("Unknown function called.")
else:
    print("No function was called by the model.")


A function call was made.
Getting weather for Houston, TX in fahrenheit...
The weather in Houston, TX is sunny and 72 degrees.


In [1]:
import openai
import os
import requests  # Assuming you'll use an API for weather data



def get_current_weather(location, unit="fahrenheit"):
    # Replace with your actual API call and processing logic
    api_key = "xxxxxxxxx"  # Replace with your API key
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units={'imperial' if unit == 'fahrenheit' else 'metric'}"
    response = requests.get(url)
    
    if response.status_code != 200:
        # Handle error
        print(f"Error fetching weather data: {response.status_code}")
        return "Error: Could not get weather data."  # Or raise an exception
    
    data = response.json()
    if data["cod"] != 200:
        return "Error fetching weather data."
    temp = data["main"]["temp"]
    description = data["weather"][0]["description"]
    return f"The current weather in {location} is {temp} degrees and {description}."

# Get API key from environment variable
api_key = os.environ.get("OPENAI_API_KEY")

# Check if API key is available
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

client = openai.OpenAI(api_key=api_key) 

tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    }
  }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = client.chat.completions.create(
  model="gpt-4-turbo-preview",
  messages=messages,
  tools=tools,
)

# Extract the function call details
tool_call = completion.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
arguments_json = tool_call.function.arguments  # Get the arguments as JSON string

# Convert arguments from JSON string to dictionary
import json
arguments = json.loads(arguments_json)  

# Execute the function with the provided arguments
if function_name == "get_current_weather":
    weather_result = get_current_weather(**arguments)  # Now arguments is a dictionary
    print(weather_result)
else:
    print("Unknown function called.")

Error fetching weather data: 401
Error: Could not get weather data.
